In [4]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [9]:
def read_files(movies_path,ratings_path):
    movies_df = pd.read_csv(movies_path,usecols=['movieId','title'],dtype={'movieId': 'int32', 'title': 'str'})
    rating_df=pd.read_csv(ratings_path,usecols=['userId', 'movieId', 'rating'],
        dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})


    return movies_df,rating_df

movies_df,ratings_df = read_files(movies_path="./Data/movies.csv",ratings_path="./Data/ratings.csv")


In [15]:
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [14]:
ratings_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [34]:
def merge_dataframes(first_data_frame, second_data_frame,on_key):
    merge_dataframe = pd.merge(second_data_frame,first_data_frame,on = on_key)
    return merge_dataframe
merged_dataframe = merge_dataframes(movies_df,ratings_df,on_key='movieId')
merged_dataframe.head() 

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,1,3,4.0,Grumpier Old Men (1995)
2,1,6,4.0,Heat (1995)
3,1,47,5.0,Seven (a.k.a. Se7en) (1995)
4,1,50,5.0,"Usual Suspects, The (1995)"


In [25]:
null_counts = merged_dataframes.isnull().sum()
print(null_counts)

userId     0
movieId    0
rating     0
title      0
dtype: int64


In [36]:
def dataframe_count_of_ratings(df,first_key,second_key ):
    movie_ratingCount = (df.
         groupby(by = [first_key])[second_key].
        count().reset_index()
    )
    movie_ratingCount = movie_ratingCount.rename(columns = {'rating': 'Number_of_Ratings'})
    return movie_ratingCount
ratings_count_df = dataframe_count_of_ratings(merged_dataframe,first_key="title",second_key="rating")
ratings_count_df

,title,Number_of_Ratings
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2
...,...,...
9714,eXistenZ (1999),22
9715,xXx (2002),24
9716,xXx: State of the Union (2005),5
9717,¡Three Amigos! (1986),26


In [37]:
final_dataframe = merge_dataframes(merged_dataframe ,ratings_count_df,on_key='title')
final_dataframe

,title,Number_of_Ratings,userId,movieId,rating
0,'71 (2014),1,610,117867,4.0
1,'Hellboy': The Seeds of Creation (2004),1,332,97757,4.0
2,'Round Midnight (1986),2,332,26564,3.5
3,'Round Midnight (1986),2,377,26564,3.5
4,'Salem's Lot (2004),1,345,27751,5.0
...,...,...,...,...,...
100831,¡Three Amigos! (1986),26,555,2478,3.0
100832,¡Three Amigos! (1986),26,561,2478,4.0
100833,¡Three Amigos! (1986),26,597,2478,3.0
100834,¡Three Amigos! (1986),26,599,2478,2.5


In [39]:
def drop_threshold_popularity(dataframe,column_name ,threshold = 200):
    dataframe = dataframe[dataframe[column_name]>= threshold]
    return dataframe
final_dataframe = drop_threshold_popularity(final_dataframe,"Number_of_Ratings" ,threshold = 200)
final_dataframe 
final_dataframe.shape
 

(4538, 5)

In [43]:
def pivot_table(df,index, columns ,values):
    df_pivot = df.pivot_table(index = index,columns= columns, values= values)
    df_pivot = df_pivot.fillna(0)

    return df_pivot

df_pivot = pivot_table(final_dataframe,index = 'title',columns='userId',values='rating')
df_pivot.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
American Beauty (1999),5.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,0.0,1.0,...,0.0,0.0,5.0,0.0,0.0,4.5,3.0,5.0,0.0,3.5
Apollo 13 (1995),0.0,0.0,0.0,0.0,3.0,4.0,4.5,4.0,0.0,0.0,...,0.0,4.0,0.0,0.0,5.0,0.0,5.0,2.0,3.0,0.0
Braveheart (1995),4.0,0.0,0.0,0.0,4.0,5.0,0.0,3.0,0.0,0.0,...,0.0,5.0,1.0,3.0,3.0,3.5,5.0,4.0,3.0,4.5
Fight Club (1999),5.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.5,...,5.0,0.0,4.0,0.0,0.0,5.0,0.0,5.0,0.0,5.0
Forrest Gump (1994),4.0,0.0,0.0,0.0,0.0,5.0,5.0,3.0,0.0,3.5,...,0.0,3.0,3.0,0.0,3.0,4.0,0.0,3.0,4.0,3.0


In [44]:
def distances_model_nneigbours(df):
    df_csr_matrix = csr_matrix(df.values)
    model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
    model_knn.fit(df_csr_matrix)
    return model_knn

model = distances_model_nneigbours(df_pivot)

In [54]:
df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,1,3,4.0,Grumpier Old Men (1995)
2,1,6,4.0,Heat (1995)
3,1,47,5.0,Seven (a.k.a. Se7en) (1995)
4,1,50,5.0,"Usual Suspects, The (1995)"


In [58]:
def operating_model(movie_vector, n_neighbours):
    distances, indices = model.kneighbors(movie_vector, n_neighbors = n_neighbours )
    return distances, indices

number_of_movies = df_pivot.shape[0]
movieId =  np.random.choice(number_of_movies)
movie_vector = df_pivot.iloc[movieId,:].values.reshape(1, -1)
distances, indices = operating_model(movie_vector, n_neighbours =4 )

In [68]:
def final_output(distances, inidces,movieId):
    print(f"The recommendation for the movie{df_pivot.index[movieId]}")
    for i in range(1, len(distances)):
        print(f" The: {i} recommendation is:{df_pivot.index[indices[i]]} with distance of :{distances[i]}")
distances = distances.flatten()
indices = indices.flatten()
final_output(distances,indices ,movieId )


The recommendation for the movieShawshank Redemption, The (1994)
 The: 1 recommendation is:Forrest Gump (1994) with distance of :0.28700655698776245
 The: 2 recommendation is:Pulp Fiction (1994) with distance of :0.29763418436050415
 The: 3 recommendation is:Silence of the Lambs, The (1991) with distance of :0.35293471813201904
